Etwas styling ...

In [ ]:
%%html
<style>
h1 { color: rgb(226, 0, 26); }
h2 { color: rgb(0, 80, 120); }
</style>

# BigData - DBWeather
---
*Programmentwurf - Wintersemester 2022 - DHBW Mannheim*\
Daniel Czeschner, Kai Grübener, Frederik Wolter

## 0. Inhalt

1. Business Understanding

2. Data Understanding

3. Data Preperation

4. Modeling

5. Evaluation

6. Deployment

*... in Anlehnung an CRISP-DM*

## 1. Business Understanding

* __Perspektive__: Deutsche Bahn

* __Vermutung__: 

<center><i>Klimaanlagen sorgen für Störungen bei extremen Temperaturen, besonders bei ICE's</i></center>

* __Ziel__: Erhebung & Auswertung von Daten zu Zugverspätungen & -ausfällen
    * Grundlegende statistische Darstellung von Fahrplan-Änderungs-Daten    
    * Identifizieren von Problemen und Ableiten möglicher Problemursachen
    * Erhebung von Wetterdaten für die Analyse einer potentiellen Korrelation

## 2. Data Understanding

Verwendete Datenquellen:
  * Deutsche Bahn Timetable API<br> 
    `apis.deutschebahn.com/db-api-marketplace/apis/timetables/`
  
  * Deutsche Bahn Reiseauskunft API<br>
    `reiseauskunft.bahn.de/bin/`
  
  * BrightSky API<br>
    `api.brightsky.dev/`

## 2.1 Data Understanding – DB Timetable

* API des Deutschen Bahn API-Marketplace

* `/station/{pattern}`
```xml
<station meta="518342" name="Mannheim Hbf" eva="8000244" p="..." ...="" />
```

* `/fchg/{evaNo}`
  * stellt viele schlecht dokumentiert Daten über Fahrplan-Änderungen bereit
    * Zugtyp, Zugnummer
    * Durchsagen? 
    * **geänderte** Gleise, Halte, Ankunf, Abfahrt, Ausfall
    <div class="alert alert-danger">
        <b>Achtung:</b> keine/kaum Plan-Daten für Ankunf und Abfahrt > andere API!
    </div>

## 2.2 Data Understanding – DB Reiseauskunft

* API der Deutschen Bahn – _"not so official"_

* `bhftafel.exe/dn?L=vs_java&start=yes&boardType={arr|dep}&input={evaNo}`

    ```
    8000244 Mannheim Hbf
    17:23
    IC  2217
    + 35
    17:24
    S      1
    no
    17:31
    ICE  611
    cancel
    ```

## 2.3 Data Understanding – BrightSky

* API des Deutschen Wetterdienstes

* `/weather?lat={Latitude}&lon={Longitude}&date={Date}&tz={TimeZone}`

    ```json
    "weather": [{ "timestamp": "2022-11-03T00:00:00+01:00",
                  "source_id": 184401,
                  "pressure_msl": 1021.7, "sunshine": 0.0,
                  "temperature": 6.8,     "condition": "dry"
                  "wind_direction": 220,  "wind_speed": 4.3,
                  "cloud_cover": 100,     "visibility": 31110,
                  "relative_humidity": 96, ...
                }, ... ],
    ```
    ```json
    "sources": [ ... ]
    ```

## 3. Data Preparation

__Abspeichern der Daten__

* Welche Daten speichern?

* Wie Daten speichern?

* In welchem Format Daten speichern?

## 3.1 Data Preparation – Deutsche Bahn

* Speichern als einzelne Ankunft/Abfahrt

* Hinzufügen von `eva` und `board_type`

* Parsen der Informationen zu `JSON`

    ```json
    {
        "timestamp": "2022-11-03T17:48:00+01:00",
        "eva": 8000105,
        "board_type": "arr",
        "con_line": "725",
        "con_type": "ICE",
        "delay": 26
    }
    ```
    
<div class="alert alert-block alert-info">
    <b>Info:</b> <code>delay</code> "cancel" wird als <code>-1</code> dargestellt
</div>
    

## 3.2 Data Preparation – BrightSky

* Speichern als einzelne stündliche Datensätze

* Hinzufügen von `lon` und `lat`
    <pre>
    <code class="cm-s-ipython language-json" style="max-height: none;">  
    { 
        <span class="cm-string cm-property">"timestamp"</span>: <span class="cm-string">"2022-11-25T14:00:00+01:00"</span>,
        <span class="cm-string cm-property">"lat"</span>: <span class="cm-number">50.05</span>,        
        <span class="cm-string cm-property">"lon"</span>: <span class="cm-number">8.6</span>,
        <span class="cm-string cm-property">"cloud_cover"</span>: <span class="cm-number">88</span>,   
        <span class="cm-string cm-property">"condition"</span>: <span class="cm-string">"dry"</span>,
        <span class="cm-string cm-property">"pressure_msl"</span>: <span class="cm-number">1018.8</span>,
        <span class="cm-string cm-property">"relative_humidity"</span>: <span class="cm-number">83</span>,
        <span class="cm-string cm-property">"sunshine"</span>: <span class="cm-number">32</span>,
        <span class="cm-string cm-property">"temperature"</span>: <span class="cm-number">9.2</span>,
        <span class="cm-string cm-property">"wind_speed"</span>: <span class="cm-number">10.1</span>, 
        <span class="cm-meta">...</span> 
    }
    </code>
    </pre>

## 4. Modeling

Die Pipeline ist wie folgt aufgebaut:

<center><img src="img/DataPipeline.png" /></center>

## 4.X Modeling – Python

// TODO

## 4.X Modeling – Datenbank

// weiß nicht wie weit wir hier ins detail gehen sollten, ist ja theoretisch auch alles lokal möglich

// Collections

* Die MongoDB wurde so konfiguriert, dass eine Authentifizierung der Clients notwendig ist.
 * Dadurch wird die Datenbank vor fremder Einsicht geschützt und konnte somit global verfügbar gemacht werden.
 * Zudem wurde ein Web-Dashboard aufgesetzt, worüber die gepspeicherten Daten einfach eingesehen werden können:
 
<center><img src="img/MongoDBDashboard.png" width="927" height="246"/></center>

## 4.X Modeling – Ausfürhung

* Python-Skripte werden durch einen Debian-Server via `cronjob` ausgeführt
  * zur _halben_ und _vollen Stunde_
  
* Bei der Ausführung wird eine Log-Datei auf dem Server erstellt:


<center>
<span class="noError" id="MathJax-Span-87" style="display: inline-block; text-align: left; font-size: 0.8em">
2022-11-03 19:00:01,408 INFO&nbsp;&nbsp;&nbsp;&nbsp;weather: Start main_weather execution ...<br>
2022-11-03 19:00:01,410 INFO&nbsp;&nbsp;&nbsp;&nbsp;train: Start main_train execution ...<br>
2022-11-03 19:00:01,708 INFO&nbsp;&nbsp;&nbsp;&nbsp;weather: Result lat=50.05 lon=8.6 has 25 elements<br>
2022-11-03 19:00:01,824 INFO&nbsp;&nbsp;&nbsp;&nbsp;weather: Result lat=49.5 lon=8.48 has 25 elements<br>
2022-11-03 19:00:01,844 INFO&nbsp;&nbsp;&nbsp;&nbsp;weather: finished: 0 inserted, 2 updated, 48 unchanged<br>
2022-11-03 19:00:01,844 INFO&nbsp;&nbsp;&nbsp;&nbsp;weather: ###########################################<br>
2022-11-03 19:00:01,917 INFO&nbsp;&nbsp;&nbsp;&nbsp;train: Result eva=8000105 type=arr has 690 lines, calc 230.0 sets<br>
2022-11-03 19:00:02,493 INFO&nbsp;&nbsp;&nbsp;&nbsp;train: Result eva=8000105 type=dep has 672 lines, calc 224.0 sets<br>
2022-11-03 19:00:02,938 INFO&nbsp;&nbsp;&nbsp;&nbsp;train: Result eva=8000244 type=arr has 291 lines, calc 97.0 sets<br>
2022-11-03 19:00:03,233 INFO&nbsp;&nbsp;&nbsp;&nbsp;train: Result eva=8000244 type=dep has 264 lines, calc 88.0 sets<br>
2022-11-03 19:00:03,301 INFO&nbsp;&nbsp;&nbsp;&nbsp;train: finished: 295 inserted, 119 updated, 225 unchanged<br>
2022-11-03 19:00:03,301 INFO&nbsp;&nbsp;&nbsp;&nbsp;train: ###########################################<br>
</span>
<center>
    


## 4.X Modeling – Auswertung

* Für das Auswerten und Visualisieren der Daten wird dieses `Jupyter Notebook` verwendet
* Für die Präsentation wird die Extension [`Rise`](https://rise.readthedocs.io/en/stable/) verwendet. 

## 5. Evaluation

* Hauptteil: Beschreibung und Erläuterung der erstellten Datenpipeline
* Wurde eine spezielle Konfiguration verwendet?
* Darstellung der Ergebnisse mit einfacher Visualisierung (z.B. Jupyter oder Streamlit)

* Tortendiagramm Zugarten
* Ausfälle allgemein
* Ausfälle pro rug art 
* Verspätungen allgemein
* Verspätungen je Zug art
* Walk of Fame Verspätungen
* Wetter kombis
    * temp <> verspätung (Zugart)
    * evtl. mehr

In [ ]:
from pandas import DataFrame
from database import Database

database = Database()
data_train = database.mongo_data_train
data_weather = database.mongo_data_weather

In [ ]:
# plot config
import matplotlib as mpl
mpl.rcParams['figure.dpi'] = 200
mpl.rcParams["figure.autolayout"] = True
mpl.rcParams["figure.figsize"] = [10.00, 4.00]

## 5.X Evaluation – Wetter Einführung

In [ ]:
data_weather.count_documents({})

In [ ]:
result = data_weather.aggregate([
    {"$group":  { "_id": { "$dateToString": { "format": "%d.%m.%Y", "date": {"$toDate": "$timestamp"} } }, "count": {"$sum": 1} } },
    {"$sort": {"_id": 1}}
])
plot = DataFrame(list(result)).plot(x='_id', y='count')

## 5.X Evaluation – Wetter

In [ ]:
result = data_weather.find({"lat": 49.5, "lon": 8.48})
plot = DataFrame(list(result)).plot(x='timestamp', y='temperature')

In [ ]:
result = data_weather.find({"lat": 50.05, "lon": 8.6})
plot = DataFrame(list(result)).plot(x='timestamp', y='temperature')

In [ ]:
result  = data_weather.find({"lat": 49.5, "lon": 8.48})
result2 = data_weather.find({"lat": 50.05, "lon": 8.6})
plot = DataFrame(list(result) + list(result2)).plot(x='timestamp', y='temperature')

## 6. Deployment

?
* Zusammenfassung: Beantwortung der Fragen aus Anwendungssicht
  * noch nicht genut Daten für zuverlässige Aussage
* (was hätte man anders machen können?)

## 6. Deployment ( Daniel 🙂 )

### Probleme
* Der Server ist am 20.11.2022 eingefroren aus unbekannter Ursache.
* Dies wurde unglücklicherweise erst ungefähr zwei Tage Später festgestellt.
 * Die Wetter-Daten konnten hier nachträglich geladen werden.
 * Die DB-Daten jedoch nicht, da die API nicht die Daten der letzten Tage bereitstellt.

# Temp Test 

## Wetter Test 

In [ ]:
agg_result = database.mongo_data_weather.aggregate([{
    "$group": {
        "_id": "$timestamp",
        "count": {"$sum": 1}
    }
}])
weather = pd.DataFrame(list(agg_result))
#weather
plot = weather.plot(x='_id', y='count')

In [ ]:
result = database.mongo_data_weather.find({})
df = pd.DataFrame(list(result))
df

In [ ]:
df.plot(x='timestamp', y='temperature')

## Züge

In [ ]:
database.mongo_data_train.count_documents(filter={})

In [ ]:
for i in database.mongo_data_train.find({"eva": 8000244}).limit(5):
    print(i)

In [ ]:
agg_result2 = database.mongo_data_train.aggregate([{
    "$group": {
        "_id": "$timestamp",
        "count": {"$sum": 1}
    }
}])
#for i in agg_result:
#    print(i)
train = pd.DataFrame(list(agg_result2))
#train
train.plot(x='_id', y='count')

In [ ]:
agg_result4 = database.mongo_data_train.aggregate([
    {
        "$group": {
            "_id": {
                "$dateToString": {
                    "format": "%d.%m.%Y",
                    "date": {"$toDate": "$timestamp"}
                }
            },
            "count": {"$sum": 1}
        }
    },
    {"$sort": {"_id": 1}}
])
#for i in agg_result:
#    print(i)
train2 = pd.DataFrame(list(agg_result4))
#train2
train2.plot(x='_id', y='count')

In [ ]:
train2

In [ ]:
for i in database.mongo_data_train.find(sort=[("delay", -1)]).limit(20):
    print(i)

In [ ]:
for i in database.mongo_data_train.find(filter={"con_type":"RE"}, sort=[("delay", -1)]).limit(20):
    print(i)

In [ ]:
timerange = database.mongo_data_train.find({
    "timestamp": {
        "$gt": "2022-11-05T17:45",
        "$lt": "2022-11-05T18:00",
    }
})
l = list(timerange)
for i in l:
    print(i)
print(len(l))

In [ ]:
database.close()

In [ ]:
# use https://stackoverflow.com/a/33079582/13777031

# https://rise.readthedocs.io/en/stable/customize.html

# https://ourcodingclub.github.io/tutorials/pandas-time-series/